In [1]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup 
import pandas as pd
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.maximize_window()
driver.implicitly_wait(10)
driver.set_page_load_timeout(5)

In [2]:
df = pd.read_csv('match_results.csv')
link = df.sample(1)
link2 = link['score_card_link'].iloc[0]
driver.get(link2)

try:
    junk_click_box = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH,'//*[@id="wzrk-cancel"]'))
    )
    junk_click_box.click()
except NoSuchElementException:
    print('No Such element')

In [9]:
def process_batting_table(xpath):
    batting = driver.find_element(By.XPATH,xpath)
    batting_outer_html = batting.get_attribute('outerHTML')
    batting = pd.read_html(batting_outer_html)[0] 
    batting = (batting
                .dropna(axis = 0, how = 'all')
                .dropna(axis = 1, how = 'all')
                .rename(columns = {'Unnamed: 1':'Dismissal'})
    )
    # last 4 elements are junk and delete the last table
    batting = batting.iloc[:-4].dropna(axis = 1, how = 'all')
    batting_soup = BeautifulSoup(batting_outer_html)

    player_links = batting_soup.find_all(class_ = 'ds-inline-flex ds-items-start ds-leading-none') 

    player_links = [a['href'] for a in player_links]

    batting['player_links'] = player_links[:batting.shape[0]]
    
    return batting 


def process_bowling_table(xpath):
    bowling = driver.find_element(By.XPATH,xpath)
    bowling_outer_html = bowling.get_attribute('outerHTML')
    bowling = pd.read_html(bowling_outer_html)[0] 
    bowling = bowling[~bowling['BOWLING'].str.contains('\d', regex = True)]
    bowling = (bowling
                .dropna(axis = 0, how = 'all')
                .dropna(axis = 1, how = 'all')
    )

    bowling_soup = BeautifulSoup(bowling_outer_html)

    player_links = bowling_soup.find_all(class_ = 'ds-inline-flex ds-items-start ds-leading-none') 

    player_links = [a['href'] for a in player_links]

    bowling['player_links'] = player_links[:bowling.shape[0]]

    return bowling


In [6]:
team_one_batting_xpath = "(//table[@class='ds-w-full ds-table ds-table-md ds-table-auto  ci-scorecard-table'])[1]"
team_two_batting_xpath = "(//table[@class='ds-w-full ds-table ds-table-md ds-table-auto  ci-scorecard-table'])[2]"
team_one_bowling_xpath = '(//table[@class="ds-w-full ds-table ds-table-md ds-table-auto "])[1]'
team_two_bowling_xpath = '(//table[@class="ds-w-full ds-table ds-table-md ds-table-auto "])[2]'

In [7]:
# process batting 
batting_one_df = process_batting_table(team_one_batting_xpath)
batting_two_df = process_batting_table(team_two_batting_xpath)
batting_one_df['team_inning'] = link['team_one'].iloc[0]
batting_two_df['team_inning'] = link['team_two'].iloc[0]
batting_one_df['match_id'] = link['score_card'].iloc[0]
batting_two_df['match_id'] = link['score_card'].iloc[0]


In [10]:
# process bowling 
bowling_one_df = process_bowling_table(team_one_bowling_xpath)
bowling_two_df =process_bowling_table(team_two_bowling_xpath)
bowling_one_df['team_inning'] = link['team_one'].iloc[0]
bowling_two_df['team_inning'] = link['team_two'].iloc[0]
bowling_one_df['match_id'] = link['score_card'].iloc[0]
bowling_two_df['match_id'] = link['score_card'].iloc[0]

In [11]:
bowling_one_df

,BOWLING,O,M,R,W,ECON,0s,4s,6s,WD,NB,player_links,team_inning,match_id
0,Ben Stokes,2.2,0,8,1,3.42,7,0,0,0,0,/player/ben-stokes-311158,England,T20I # 1846
2,Chris Woakes,3,0,41,0,13.66,4,6,1,2,0,/player/chris-woakes-247235,England,T20I # 1846
3,Mark Wood,4,0,34,3,8.50,13,4,1,2,0,/player/mark-wood-351588,England,T20I # 1846
5,Sam Curran,3,0,31,2,10.33,7,1,2,0,0,/series/icc-men-s-t20-world-cup-2022-23-129813...,England,T20I # 1846
7,Adil Rashid,4,0,24,0,6.00,6,1,0,0,0,/player/sam-curran-662973,England,T20I # 1846
8,Liam Livingstone,3,0,17,3,5.66,5,1,0,0,0,/player/adil-rashid-244497,England,T20I # 1846
